# MON NAIVE BAYES

In [19]:
%pip install spacy nltk
!python -m spacy download en_core_web_sm

  Obtaining dependency information for spacy from https://files.pythonhosted.org/packages/f5/8f/96f8f02c4719ad48973eb509b2bdaf7afe68447b54cb30f00420032c9c12/spacy-3.7.2-cp310-cp310-win_amd64.whl.metadata
  Obtaining dependency information for spacy-loggers<2.0.0,>=1.0.0 from https://files.pythonhosted.org/packages/33/78/d1a1a026ef3af911159398c939b1509d5c36fe524c7b644f34a5146c4e16/spacy_loggers-1.0.5-py3-none-any.whl.metadata
  Obtaining dependency information for murmurhash<1.1.0,>=0.28.0 from https://files.pythonhosted.org/packages/ed/9d/d62d12e3ecc6f99eddea6289413669a905d2ebb15cf9fe075336ca6cceaa/murmurhash-1.0.10-cp310-cp310-win_amd64.whl.metadata
  Obtaining dependency information for cymem<2.1.0,>=2.0.2 from https://files.pythonhosted.org/packages/51/12/4aa9eec680c6d12b2275d479e159c3d063d7c757175063dd45386e15b39d/cymem-2.0.8-cp310-cp310-win_amd64.whl.metadata
  Obtaining dependency information for preshed<3.1.0,>=3.0.2 from https://files.pythonhosted.org/packages/f3/72/108426ca3b6


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: C:\Users\nicos\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.1/12.8 MB 2.3 MB/s eta 0:00:06
      --------------------------------------- 0.2/12.8 MB 2.9 MB/s eta 0:00:05
     - -------------------------------------- 0.4/12.8 MB 3.2 MB/s eta 0:00:04
     -- ------------------------------------- 0.7/12.8 MB 4.0 MB/s eta 0:00:04
     --- ------------------------------------ 1.0/12.8 MB 4.7 MB/s eta 0:00:03
     --- ------------------------------------ 1.2/12.8 MB 4.3 MB/s eta 0:00:03
     ---- ----------------------------------- 1.3/12.8 MB 4.1 MB/s eta 0:00:03
     ---- ----------------------------------- 1.4/12.8 MB 3.9 MB/s eta 0:00:03
     ---- ----------------------------------- 1.5/12.8 MB 3.8 MB/s eta 0:00:03
     ----- ---------------------------------- 1.6/12.8 MB 3.6 MB/s eta 0:00:04
     ----- ---------------------------------- 1.8/12.8 MB 3.5 MB/s eta 0:00:04
     ----- ---------------------------------- 1.9/12.8 MB 3


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: C:\Users\nicos\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [20]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords, words
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import spacy
import string
from spacy.lang.en import English
from spacy.tokens import Doc
from spacy.tokenizer import Tokenizer
from spacy.lang.en.stop_words import STOP_WORDS
from nltk.stem import LancasterStemmer

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('words')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nicos\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nicos\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\nicos\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\words.zip.


True

In [119]:
df = pd.read_excel('./goemotions_simplified.xlsx')

print(df.shape)
df.head()

(70000, 2)


,text,emotion
0,That game hurt.,sadness
1,>sexuality shouldn’t be a grouping category I...,NaN
2,"You do right, if you don't care then fuck 'em!",neutral
3,Man I love reddit.,love
4,"[NAME] was nowhere near them, he was by the Fa...",neutral


Data Preparation

In [120]:
# Hapus data yg kosong
df.dropna(inplace = True)

# Hapus data duplikat
df.drop_duplicates(inplace = True)

# Hapus data yg tidak memiliki emosi
for x in df.index:
  if df.loc[x, "emotion"] == "NaN":
    df.drop(x,inplace=True)

df.shape

(59935, 2)

Tokenizing

In [121]:
# Initialize the Lancaster Stemmer from NLTK
stemmer = PorterStemmer()

# Set of valid words from NLTK's words corpus
# valid_words = set(words.words())

# Function to clean text, perform stemming, and filter words not in the corpus
def clean_text(text):
    # Convert to lowercase
    text = text.lower()
    
    # Remove URLs
    text = re.sub(r'http\S+', '', text)

    # Replace non-alphanumeric characters with a single space
    text = re.sub(r'[^a-zA-Z0-9@]', ' ', text)
    
    # remove @ mention 
    # Split the text into words
    words = text.split()

    # Filter out words that start with '@'
    filtered_words = [word for word in words if not word.startswith('@')]

    # Join the filtered words back into a string
    text = ' '.join(filtered_words)

    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Remove numbers
    text = re.sub(r'\d+', '', text)

    # Tokenize the text
    words = nltk.word_tokenize(text)

    # Filter words not in the corpus
    # cleaned_words = [word for word in words if word in valid_words]

    # Remove stopwords and perform stemming
    stemmed_words = [stemmer.stem(word) for word in words if word not in stopwords.words('english') and '@' not in word]
    
    cleaned_text = ' '.join(stemmed_words)
    
    return cleaned_text
# Apply the cleaning function to the 'content' column
df['cleaned_text'] = df['text'].apply(clean_text)

# Display the cleaned dataset
# df.head()
print(df[['text', 'cleaned_text']].head())

                                                text  \
0                                    That game hurt.   
2     You do right, if you don't care then fuck 'em!   
3                                 Man I love reddit.   
4  [NAME] was nowhere near them, he was by the Fa...   
5  Right? Considering it’s such an important docu...   

                                        cleaned_text  
0                                          game hurt  
2                                 right care fuck em  
3                                    man love reddit  
4                            name nowher near falcon  
5  right consid import document know damn thing b...  


In [106]:


# df.drop("id", axis=1, inplace=True)
# df.drop("author", axis=1, inplace=True)
# df.drop("subreddit", axis=1, inplace=True)
# df.drop("link_id", axis=1, inplace=True)
# df.drop("parent_id", axis=1, inplace=True)
# df.drop("created_utc", axis=1, inplace=True)
# df.drop("rater_id", axis=1, inplace=True)
# df.drop("example_very_unclear", axis=1, inplace=True)

# df = df.melt(id_vars=['text'], var_name='emotion', value_name='value')
# df = df[df['value'] == 1]
# df = df.drop(columns='value')
# print(df.head())

# df.to_csv('cleaned_goemotions.csv', index=False)

In [24]:
for x in df.index:
  if df.loc[x, "sentiment"] == NaN:
    df.drop(x,inplace=True)

In [25]:
df.head()

,tweet_id,sentiment,content,cleaned_content
1,1956967666,sadness,Layin n bed with a headache ughhhh...waitin o...,layin n bed headach ughhhh waitin call
2,1956967696,sadness,Funeral ceremony...gloomy friday...,funer ceremoni gloomi friday
3,1956967789,enthusiasm,wants to hang out with friends SOON!,want hang friend soon
4,1956968416,neutral,@dannycastillo We want to trade with someone w...,want trade someon houston ticket one
5,1956968477,worry,Re-pinging @ghostridah14: why didn't you go to...,ping go prom bc bf like friend


In [223]:
%pip install BeautifulSoup4 scikit-learn textblob 
import requests
from bs4 import BeautifulSoup
import pandas as pd
from textblob import TextBlob
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: C:\Users\nicos\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Multinomial NB

In [300]:
# Load the training data from the CSV file
df_train_multinomial = df

# Preprocess the training data
x = df_train_multinomial['cleaned_text']
y = df_train_multinomial['emotion']

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=125)

# Make model training pipeline
clf = make_pipeline(
    TfidfVectorizer(),
    MultinomialNB(fit_prior=True),
)

# Fit the model to the training data
clf.fit(X_train, y_train)

# scores = cross_val_score(clf, x, y, cv=5)
# print('Cross-validation accuracy: ', scores.mean())

# Predict on the test data
clf.fit(x, y)
y_pred = clf.predict(X_test)
# print(y_pred)
# print(y_test)
# Evaluate the accuracy of the classifier
# print(classification_report(y_test, y_pred))
print(metrics.accuracy_score(y_test, y_pred))
# accuracy = metrics.accuracy_score(y_test, y_pred)
# print(f"Accuracy: {accuracy}")


0.28097105197297073


K Nearest Neighbors

In [302]:
# Load the training data from the CSV file
df_train_multinomial = df

# Preprocess the training data
x = df_train_multinomial['cleaned_text']
y = df_train_multinomial['emotion']

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.05, random_state=42)

# Make model training pipeline
clf = make_pipeline(
    TfidfVectorizer(), 
    KNeighborsClassifier(n_neighbors=5, weights='distance'),
)

# Fit the model to the training data
clf.fit(X_train, y_train)
# clf.fit(x, y)

# scores = cross_val_score(clf, X_test, y_test, cv=5)
# print('Cross-validation accuracy: ', scores)

# Predict on the test data
y_pred = clf.predict(X_test)
# print(y_pred)
# print(y_test)
# Evaluate the accuracy of the classifier
# print(classification_report(y_test, y_pred))
print(metrics.accuracy_score(y_test, y_pred))


0.16182849516182848


Random Forest Classifier

In [315]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from functools import reduce

def generate_rf(X_train, y_train, X_test, y_test):
    # rf = make_pipeline(
    vectorizer = TfidfVectorizer()
    vectorized = vectorizer.fit_transform(X_train)
    rf = RandomForestClassifier(n_estimators=7, min_samples_leaf=3, random_state=42)
    rf.fit(vectorized, y_train)
    return rf

def combine_rfs(rf_a, rf_b):
    rf_a.estimators_ += rf_b.estimators_
    rf_a.n_estimators = len(rf_a.estimators_)
    return rf_a

df_train_multinomial = df

# Preprocess the training data
x = df_train_multinomial['cleaned_text']
y = df_train_multinomial['emotion']
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=125)


rfs = [generate_rf(X_train, y_train, X_test, y_test) for i in range(10)]
rf_combined = reduce(combine_rfs, rfs)

vectorizer = TfidfVectorizer()
vectorized = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

y_pred = rf_combined.predict(X_test)

print(metrics.accuracy_score(y_test, y_pred))

0.3457078501710186


In [44]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB

# Load the training data from the CSV file
df_train_gaussian = df

# Preprocess the training data
x = df_train_gaussian['cleaned_content']
y = df_train_gaussian['sentiment']

# using count vectorizer
vectorizer = CountVectorizer(preprocessor=lambda x: x, tokenizer=lambda x: x)
x_train_vectorized = vectorizer.fit_transform(x)

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(x_train_vectorized, y, test_size=0.2, random_state=42)

# Initialize the Gaussian Naive Bayes classifier
gnb = GaussianNB()

# Train the classifier on the training data
gnb.fit(X_train.toarray(), y_train)

# Predict on the test data
y_pred = gnb.predict(X_test.toarray())

# Evaluate the accuracy of the classifier
accuracy = metrics.accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")


C:\Users\nicos\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Accuracy: 0.22399489470325462


In [ ]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

titles = ["magics-return-i-can-see-the-spirits", "taking-the-mafia-to-the-magic-world", "ill-surpass-the-mc"]

for title in titles:
    url = f'https://www.lightnovelworld.com/novel/{title}'
    response = requests.get(url, headers=headers)

    data_dict = {'title': '', 'author': '', 'rating': '', 'categories': [], 'comments': [], 'sentiments': []}

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')

        # Get novel title
        title_element = soup.find('h1', class_='novel-title')
        if title_element:
            data_dict['title'] = title_element.text.strip()

        # Get author information
        author_element = soup.find('span', attrs={'itemprop': 'author'})
        if author_element:
            data_dict['author'] = author_element.text.strip()

        # Get rating information
        meta_rating_element = soup.find('meta', attrs={'itemprop': 'ratingValue'})
        if meta_rating_element:
            data_dict['rating'] = meta_rating_element.get('content', '').strip()

        # Mencari semua elemen <p> di dalam <div> dengan class 'comment-text'
        comment_text_divs = soup.find_all('div', class_='comment-text')

        for comment_text_div in comment_text_divs:
            p_elements = comment_text_div.find_all('p')

            for p_element in p_elements:
                paragraph_text = p_element.text.strip()
                data_dict['comments'].append(paragraph_text)

                # Perform sentiment analysis using TextBlob
                blob = TextBlob(paragraph_text)
                sentiment = 'positive' if blob.sentiment.polarity > 0 else 'negative' if blob.sentiment.polarity < 0 else 'neutral'
                data_dict['sentiments'].append(sentiment)

        # Get Categories
        categories_header = soup.find('h4', string='Categories')
        if categories_header:
            categories_list = []
            ul_element = categories_header.find_next('ul')
            if ul_element:
                categories = ul_element.find_all('li')
                for category in categories:
                    category_text = category.text.strip()
                    categories_list.append(category_text)
                data_dict['categories'] = categories_list

        # Print each comment with its predicted sentiment
        print(data_dict)
        for comment, sentiment in zip(data_dict['comments'], data_dict['sentiments']):
            print(f"Comment: {comment}")
            print(f"Predicted Sentiment: {sentiment}\n")
    else:
        print(f"Gagal mengambil halaman web untuk judul {title}. Kode status:", response.status_code)


{'title': "Magic's Return: I Can See The Spirits", 'author': 'SleepDeprivedSloth', 'rating': '4.0', 'categories': ['Fantasy', 'Action', 'Romance', 'Comedy'], 'comments': ['Any novels with similar romance? Pls reccomend me some...', 'The novel has good romance, pacing, fight scenes, but Vaan feels like a 15 year old.\nWhat 31 year old genocidal mob boss says things like “simp general” and has the patience and ego of a toddler? His *ss would’ve gotten killed years ago.\nIf you can get past the fact the mc is basically an egotistical teenager with no character development yet, the rest of the novel is enjoyable.', 'I can’t get over his personality. He literally attacks people and when they defend themselves he says “my life is so precious how dare he try to take it from me”', 'Well I would argue that isn’t actually as bad as some other actions he takes as most gangsters use self-justification as a way to cope with the crimes they commit. I personally can’t get over how he literally acts l